# Computing inter-annotator agreement for span annotation tasks

<br><a target="_blank" href="https://colab.research.google.com/github/haukelicht/advanced_text_analysis/blob/main/notebooks/annotation/compute_ica_pledge_extraction.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Background

### Span annotations

**Span annotation** is a _sequence labeling_ task (cf. Jurafsky & Martin [2025](https://web.stanford.edu/~jurafsky/slp3/17.pdf)) that involves marking passages in a text that represent a certain concept, such as an entity.

Accordingly, span annotations recorded **character spans** that indicate which passage(s) of a text were marked (if any). 
At least this is the most granular way of representing sequence labeling annotations.
(But see the [CoNLL format](https://www.geeksforgeeks.org/nlp/what-is-conll-data-format/), that records annotations at the token level.)

The example below illustrates this for an English sentence.
The marked span is represented by a list with two elements that indicate the span's character start and end positions in the text:


In [24]:
translation = "We will restrict foreign equity holding in private television broadcasting to 20%."
span = [8, 39]

print("first character of the span:", repr(translation[span[0]]))
print("last character of the span:", repr(translation[span[1]-1]))
print("extracted span:", repr(translation[span[0]:span[1]]))

first character of the span: 'r'
last character of the span: 'g'
extracted span: 'restrict foreign equity holding'


### Computing inter-annotator agreement for sequence labeling tasks.

We want to compute to what extent annotators marked spans agree with each other.
Specifically, following Krippendorff and others, we want to compute a _chance-adjusted_ inter-annotator agreement metric that adjusts for the probability that an agreement arises by chance.

The appraoch to computing inter-coder agreement adopted in this notebook is based on Baylan et al. ([2022](https://doi.org/10.1145/3485447.3512242)).

### Setup

In [25]:
# check if on colab
COLAB = True
try:
    import google.colab
except:
    COLAB=False

if COLAB:
    # shallow clone of current state of main branch 
    !git clone --branch main --single-branch --depth 1 --filter=blob:none https://github.com/haukelicht/advanced_text_analysis.git
    # make repo root findable for python
    import sys, os
    sys.path.append(f"{os.getcwd()}/advanced_text_analysis/")

    # install required packages
    !pip install nltk==3.9.1 krippendorff==0.8.1

    # get NLTK tokenizer data
    from nltk import download
    download('punkt')

In [26]:
from pathlib import Path

import pandas as pd
import numpy as np

from nltk.tokenize import TreebankWordTokenizer

from krippendorff import alpha as k_alpha

# NOTE: you may need to adjust your PYTHONPATH so that the src module can be found
from src.annotation.agreement import InterAnnotatorAgreement, overlap_distance, split_iaa_by_item

data_path = "data/labeled/fornaciari_we_2021"
data_path = ("/content/advanced_text_analysis/" if COLAB else "../../") + data_path
data_path = Path(data_path)

## Read the annotations

In [27]:
# TODO: change `"llms"` to the name of your group's folder
annotations_path = data_path / "annotations" / "extraction" / "group1"

# list all annotation files produced by doccano 
#  (each records annotations by one annotator)
fps = list(annotations_path.glob('*.jsonl'))

# read the annoations into a long-format DataFrame
annotations = pd.concat({fp.stem: pd.read_json(fp, lines=True) for fp in fps}, ignore_index=False).reset_index(level=0, names=['annotator'])

# list unique annotators
annotations.annotator.unique().tolist()

# discard entity type
annotations['label'] = annotations.label.apply(lambda x: [anno[:2] for anno in x])

In [28]:
# add metadata and reformat the DataFrame
if 'metadata' in annotations.columns:
    metadata = annotations['metadata'].apply(pd.Series)
    metadata.drop(columns=['label'], inplace=True)
    annotations[metadata.columns] = metadata
    annotations.drop(columns=['metadata'], inplace=True)

annotations = annotations.sort_values(by=['text_id', 'annotator']).reset_index(drop=True)
annotations = annotations[['text_id', 'text', 'annotator', 'label']]
annotations['spans'] = annotations.apply(lambda x: [x['text'][lab[0]:lab[1]] for lab in x['label']], axis=1)

# count number of annotations per annotator and text
annotations['n_annos'] = annotations.label.map(len)
annotations['no_annos'] = annotations['n_annos']==0

We are interested to what extent annotators marked spans agree.
Do the words marked in a sentence by one annotator agree with those marked by the other annotators?

But note that annotated words may vary a lot in their character length.
We don't want this varation to influence our agreement measure.

So we need to convert the character-level into token-level annotations.
For this, we first tokenize the sentence and than map the character-level span start and end indexes to their correspondong token-level indexes:

In [29]:
tokenizer = TreebankWordTokenizer()

# list(tokenizer.span_tokenize(text))
def character_to_token_spans(text: str, spans: list[tuple[int, int]]) -> list[tuple[int, int]]:
    token_spans = list(tokenizer.span_tokenize(text))
    token_span_list = []
    for span in spans:
        start_char, end_char = span
        # Find the first token that starts after or at the start_char
        start_token = next((i for i, (s, _) in enumerate(token_spans) if s >= start_char), None)
        # Find the last token that ends before or at the end_char
        end_token = next((i for i, (_, e) in reversed(list(enumerate(token_spans))) if e <= end_char), None)
        if start_token is not None and end_token is not None and start_token <= end_token:
            token_span_list.append((start_token, end_token + 1))  # +1 to make it exclusive
    return token_span_list

The example below illustrates the functionality of the `character_to_token_spans()` function.

In [30]:
text = r"We will restrict foreign equity holding in private television broadcasting to 20% and prevent cross holding to avoid emergence of monopolies in the media."
# annotation
annotation = [[8, 81], [86, 153]]
print('extracted spans:', end='')
print('', *[text[slice(*span)] for span in annotation], sep='\n - ')
print()

tok_spans = character_to_token_spans(text, annotation)
print("token-level annotations:", tok_spans)
tokens = tokenizer.tokenize(text)
print()

print('tokens in extracted spans:')
for tspan in tok_spans:
    print(' - ', tokens[slice(*tspan)])

extracted spans:
 - restrict foreign equity holding in private television broadcasting to 20%
 - prevent cross holding to avoid emergence of monopolies in the media

token-level annotations: [(2, 13), (14, 25)]

tokens in extracted spans:
 -  ['restrict', 'foreign', 'equity', 'holding', 'in', 'private', 'television', 'broadcasting', 'to', '20', '%']
 -  ['prevent', 'cross', 'holding', 'to', 'avoid', 'emergence', 'of', 'monopolies', 'in', 'the', 'media']


Let's apply this logic to all the data:

In [31]:
# extract tokens
annotations['tokens'] = annotations.apply(lambda x: tokenizer.tokenize(x['text']), axis=1)
# determine token-level span informations
annotations['token_spans'] = annotations.apply(lambda x: character_to_token_spans(x['text'], x['label']), axis=1)

Let's look at an example

In [32]:
ex = annotations.loc[annotations.text_id == 604, ['text', 'annotator', 'spans', 'label', 'tokens', 'token_spans']]
print('Text:', repr(ex.text.iloc[0]))
print('Annotations:')
for _, row in ex.iterrows():
    print(' -', row.annotator+':', end='')
    print('', *row['spans'], sep='\n   - ')
    print()

Text: 'Reservations for the poor among ‘Forward Classes’ will be introduced after receiving recommendations of the Commission set up for this purpose .'
Annotations:
 - carolin.rinner:
   - Reservations for the poor among ‘Forward Classes’ will be introduced after receiving recommendations

 - ingmar.rapp:
   - Reservations for the poor among ‘Forward Classes’ will be introduced after receiving recommendations of the Commission

 - martin:
   - Reservations for the poor among ‘Forward Classes’ will be introduced 

 - quiring.paul:
   - Reservations for the poor among ‘Forward Classes’ will be introduced



## Compute inter-annotator agreement

### Sentence level

In [33]:
tmp = annotations[['annotator', 'text_id', 'no_annos']].copy()
tmp['no_annos'] = ~tmp['no_annos']
tmp = tmp.pivot(index='annotator', columns='text_id', values='no_annos').fillna(False).astype(int)
k_alpha(tmp.values, level_of_measurement='nominal')
# NOTE: just scratching the threshold of 0.67 for acceptable agreement (see https://www.sciencedirect.com/science/article/pii/S2215016123005411)

/var/folders/_c/cm0nk6y92rz2l6ct3npgw9tr0000gn/T/ipykernel_72862/1569186875.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tmp = tmp.pivot(index='annotator', columns='text_id', values='no_annos').fillna(False).astype(int)


0.49623700054734543

Krippendorff (cited in Neuendorff, 2017) names the following standards

- Rely only on variables with reliabilities above α = .800.
- Consider variables with reliabilities between α = .667 and α = .800 only for drawing tentative conclusions.

### Span level

In [34]:
# Braylan et al.'s code requires that annotators and items are identified by integer IDs
annotations['item_id'] = pd.Categorical(annotations['text_id']).codes
annotations['annotator_id'] = pd.Categorical(annotations['annotator']).codes

In [35]:
# create an agreement object with the entity distance function
iaa = InterAnnotatorAgreement(
    annotations, 
    item_colname="item_id", 
    uid_colname="annotator_id", 
    label_colname="label", 
    distance_fn=overlap_distance
)

In [36]:
# compute agreement measures
iaa.setup(parallel_calc=False)
krippendorff_alpha = iaa.get_krippendorff_alpha()
sigma_score = iaa.get_sigma()
ks_score = iaa.get_ks()

print(f"Krippendorff's Alpha: {krippendorff_alpha:.4f}")
print(f"Sigma Score: {sigma_score:.4f}")
print(f"KS Score: {ks_score:.4f}")

Krippendorff's Alpha: 0.4829
Sigma Score: 0.5509
KS Score: 0.3088


## Sentence-level disagreement analysis

We can also compute agreement at the sentence level:

In [37]:
# Get a list of single-item lists
item_ids = annotations['item_id'].unique()
split_items = [[item] for item in item_ids]

item_iaas = split_iaa_by_item(iaa, split_items)

# Now compute item-level ICA measures
per_item_ica = []
for m in item_iaas:
    # m.setup(parallel_calc=False)
    m.expected_distances = iaa.expected_distances
    try:
        per_item_ica.append({
            "item_id": m.annodf["item"].iloc[0],
            "alpha": m.get_krippendorff_alpha(),
            "sigma": m.get_sigma(),
            "ks": m.get_ks()
        })
    except Exception as e:
        print(f"Error processing item {m.annodf['item_id'].iloc[0]}: {e}")

In [38]:
item_icas = annotations[['text_id', 'text', 'item_id']].drop_duplicates().merge(pd.DataFrame(per_item_ica), on='item_id', how='left')

In [39]:
item_icas = item_icas.merge(
    annotations.groupby('item_id').agg({
        'spans': lambda spans: len(set.union(*spans.apply(set).tolist())), 
         'no_annos': 'all'
    }).rename(columns={'spans': 'n_annos'}).reset_index()
)

Let's plot the distribution of sentence-level agreement estimates:

In [40]:
from matplotlib import pyplot as plt
plt.figure(figsize=(3, 2))
item_icas.alpha.hist(bins=50)
# plt.yscale('log')
plt.xlabel("Krippendorff's alpha")
plt.ylabel("Frequency")
plt.xlim(-1, 1.05)
plt.show()

In [41]:
# what's the average sentence-level agreement across sentences?
item_icas.alpha.mean()

0.48244892641237713

In [42]:
# and in sentences with min. one annotation?
item_icas.query('no_annos == False').alpha.mean()

0.1913264475193393

In [43]:
# share with "unacceptable" agreement?
(item_icas.alpha < 0.65).mean()

0.5

#### clear cut

In [44]:
tmp = item_icas[item_icas.alpha == 1.0]
tmp = tmp.sort_values(['alpha', 'item_id'])
print(tmp.item_id.nunique())
print(tmp.item_id.nunique()/len(per_item_ica))

18
0.36


In [45]:
tab = item_icas[['item_id', 'text_id', 'n_annos']].drop_duplicates().n_annos.value_counts().sort_index().reset_index()
tab['prop'] = tab['count'] / tab['count'].sum()
tab.columns = ['n_annotations', 'count', 'proportion']
tab

,n_annotations,count,proportion
0,0,18,0.36
1,1,5,0.10
2,2,11,0.22
3,3,9,0.18
4,4,7,0.14


In [46]:
ex = item_icas[item_icas.n_annos==3].iloc[1]
print(ex.text)
print(*annotations[annotations.item_id == ex.item_id].spans.tolist(), sep='\n')

It will enable industry to access agriculture produce directly from farmers, and suggest ways to step up exports, including to organized retail outlets abroad .
['t will enable industry to access agriculture produce directly from farmers, and suggest ways to step up exports, ']
['It will enable industry to access agriculture produce directly from farmers']
[]
['will enable industry to access agriculture produce directly from farmers']


#### good(ish)

In [47]:
tmp = item_icas[np.logical_and(item_icas.alpha >= 0.65, item_icas.alpha < 1.0)]
tmp = tmp.sort_values(['alpha', 'item_id'])
print(tmp.item_id.nunique())
print(tmp.item_id.nunique()/len(per_item_ica))

7
0.14


In [48]:
items = tmp.item_id.unique()
examples = tmp[tmp.item_id.isin(items)].merge(annotations[['item_id', 'annotator', 'spans']], on='item_id', how='left')
for item, ex in examples.groupby('item_id'):
    print(f"Item: {item}, k-alpha: {ex.alpha.iloc[0]:.4f}")
    print("Text:", repr(ex['text'].iloc[0]))
    print("Spans:")
    for m, span in zip(ex['annotator'], ex['spans']):
        print(f" - {span} ({m})")
    print("\n---\n")

Item: 7, k-alpha: 0.8204
Text: 'We will also introduce a multi - purpose identity card for all citizens .'
Spans:
 - ['We will also introduce a multi - purpose identity card for all citizens .'] (carolin.rinner)
 - ['We will also introduce a multi - purpose identity card for all citizens'] (ingmar.rapp)
 - ['introduce a multi - purpose identity card for all citizens '] (martin)
 - ['We will also introduce a multi - purpose identity card for all citizens'] (quiring.paul)

---

Item: 12, k-alpha: 0.8081
Text: 'More specialist battalions will be raised and positioned in key locations across the country .'
Spans:
 - ['More specialist battalions will be raised and positioned in key locations across the country'] (carolin.rinner)
 - ['More specialist battalions will be raised and positioned in key locations across the country'] (ingmar.rapp)
 - ['More specialist battalions will be raised and positioned in key locations'] (martin)
 - ['More specialist battalions will be raised and positioned 

#### poor

In [49]:
tmp = item_icas[np.logical_and(item_icas.alpha >= 0.0, item_icas.alpha < 0.65)]
tmp = tmp.sort_values(['alpha', 'item_id'])
print(tmp.item_id.nunique())
print(tmp.item_id.nunique()/len(per_item_ica))

13
0.26


In [50]:
items = tmp.item_id.unique()
examples = tmp[tmp.item_id.isin(items)].merge(annotations[['item_id', 'annotator', 'spans']], on='item_id', how='left')
for item, ex in examples.groupby('item_id'):
    print(f"Item: {item}, k-alpha: {ex.alpha.iloc[0]:.4f}")
    print("Text:", repr(ex['text'].iloc[0]))
    print("Spans:")
    for m, span in zip(ex['annotator'], ex['spans']):
        print(f" - {span} ({m})")
    print("\n---\n")

Item: 10, k-alpha: 0.0740
Text: 'a . We will ensure the passage of the Women’s Reservation Bill .'
Spans:
 - ['We will ensure the passage of the Women’s Reservation Bill'] (carolin.rinner)
 - [] (ingmar.rapp)
 - ['ensure the passage of the Women’s Reservation Bill'] (martin)
 - ['We will ensure the passage of the Women’s Reservation Bill'] (quiring.paul)

---

Item: 11, k-alpha: 0.0892
Text: '11 . We will strengthen the legal and institutional framework to protect our children .'
Spans:
 - ['We will strengthen the legal and institutional framework to protect our children'] (carolin.rinner)
 - [] (ingmar.rapp)
 - ['strengthen the legal and institutional framework to protect our children '] (martin)
 - ['We will strengthen the legal and institutional framework to protect our children'] (quiring.paul)

---

Item: 17, k-alpha: 0.1520
Text: 'Small industry will be particularly encouraged in states and regions where the potential for large or heavy industry is limited .'
Spans:
 - [] (caroli

#### bad

In [51]:
tmp = item_icas[item_icas.alpha < 0.0]
tmp = tmp.sort_values(['alpha', 'item_id'])
print(tmp.item_id.nunique())
print(tmp.item_id.nunique()/len(per_item_ica))

12
0.24


In [52]:
items = tmp.item_id.unique()
examples = tmp[tmp.item_id.isin(items)]
# draw random sample of 3 examples
examples = examples.sample(3, random_state=42)
examples = examples.merge(annotations[['item_id', 'annotator', 'spans']], on='item_id', how='left')
for item, ex in examples.groupby('item_id'):
    print(f"Item: {item}, k-alpha: {ex.alpha.iloc[0]:.4f}")
    print("Text:", repr(ex['text'].iloc[0]))
    print("Spans:")
    for m, span in zip(ex['annotator'], ex['spans']):
        print(f" - {span} ({m})")
    print("\n---\n")

Item: 24, k-alpha: -0.3827
Text: 'The Ministry of Finance will be directed to provide all required resources to implement this program in a time - bound manner .'
Spans:
 - [] (carolin.rinner)
 - [] (ingmar.rapp)
 - ['directed to provide all required resources'] (martin)
 - ['The Ministry of Finance will be directed'] (quiring.paul)

---

Item: 39, k-alpha: -0.1306
Text: 'Make potable drinking water available to all thus reducing water - borne diseases, which will automatically translate into Diarrhoea - free India .'
Spans:
 - [] (carolin.rinner)
 - ['Make potable drinking water available to all'] (ingmar.rapp)
 - [] (quiring.paul)

---

Item: 44, k-alpha: -0.1306
Text: 'The Congress will identify those environmental management functions that could be delegated to the states and local bodies .'
Spans:
 - [] (carolin.rinner)
 - [] (ingmar.rapp)
 - ['Congress will identify those environmental management functions'] (quiring.paul)

---

